<a href="https://colab.research.google.com/github/Pigwen/hands-on-sft/blob/main/Chapter_0_TL%3BDR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U transformers bitsandbytes huggingface_hub trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.4/517.4 kB 42.5 MB/s eta 0:00:00


In [2]:
import torch
from torch import device
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float32
)

repo_id = "microsoft/Phi-3-mini-4k-instruct"
model = AutoModelForCausalLM.from_pretrained(
    repo_id,
    device_map="cuda:0",
    quantization_config=bnb_config)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [3]:
print(model.get_memory_footprint() / 1e6)

2206.341312


In [4]:
model

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear4bit(in_features=3072, out_features=9216, bias=False)
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear4bit(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLUActivation()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (norm): Phi3RMSNorm((3072,), eps=1e-05)
    (rotary_emb): Phi3RotaryEmbedding()
  )
  (lm_head): Linear(in_features=

In [5]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,
    lora_alpha=16,
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
    target_modules=["o_proj", "qkv_proj", "gate_up_proj", "down_proj"],
)

model = get_peft_model(model, config)
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Phi3ForCausalLM(
      (model): Phi3Model(
        (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
        (layers): ModuleList(
          (0-31): 32 x Phi3DecoderLayer(
            (self_attn): Phi3Attention(
              (o_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (

In [6]:
print(model.get_memory_footprint() / 1e6)

2651.074752


In [7]:
train_p, total_p = model.get_nb_trainable_parameters()
print(f"Trainable parameters: {train_p / 1e6:.2f}M")
print(f"Total parameters: {total_p / 1e6:.2f}M")
print(f"% of trainable parameters: {100 * train_p / total_p:.2f}%")

Trainable parameters: 12.58M
Total parameters: 3833.66M
% of trainable parameters: 0.33%


# Formatting Your Dataset

In [8]:
from datasets import load_dataset

dataset = load_dataset("dvgodoy/yoda_sentences", split="train")
dataset

README.md:   0%|          | 0.00/531 [00:00<?, ?B/s]

sentences.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/720 [00:00<?, ? examples/s]

Dataset({
    features: ['sentence', 'translation', 'translation_extra'],
    num_rows: 720
})

In [9]:
dataset[0]

{'sentence': 'The birch canoe slid on the smooth planks.',
 'translation': 'On the smooth planks, the birch canoe slid.',
 'translation_extra': 'On the smooth planks, the birch canoe slid. Yes, hrrrm.'}

In [10]:
def format_dataset(examples):
  if isinstance(examples["prompt"], list):
    output_texts = []
    for i in range(len(examples["prompt"])):
      converted_sample = [
          {"role": "user", "content": examples["prompt"][i]},
          {"role": "assistant", "content": examples["completion"][i]},
      ]
      output_texts.append(converted_sample)
    return {"messages": output_texts}
  else:
    return {
        "messages": [
            {"role": "user", "content": examples["prompt"]},
            {"role": "assistant", "content": examples["completion"]},
        ]
    }

In [11]:
dataset = dataset.rename_column("sentence", "prompt").rename_column("translation_extra", "completion").map(format_dataset)
dataset = dataset.remove_columns(["prompt", "completion", "translation"])
messages = dataset[0]["messages"]
messages

Map:   0%|          | 0/720 [00:00<?, ? examples/s]

[{'content': 'The birch canoe slid on the smooth planks.', 'role': 'user'},
 {'content': 'On the smooth planks, the birch canoe slid. Yes, hrrrm.',
  'role': 'assistant'}]

# Tokenizer

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(repo_id)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

tokenizer.chat_template

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

"{% for message in messages %}{% if message['role'] == 'system' %}{{'<|system|>\n' + message['content'] + '<|end|>\n'}}{% elif message['role'] == 'user' %}{{'<|user|>\n' + message['content'] + '<|end|>\n'}}{% elif message['role'] == 'assistant' %}{{'<|assistant|>\n' + message['content'] + '<|end|>\n'}}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '<|assistant|>\n' }}{% else %}{{ eos_token }}{% endif %}"

In [13]:
print(tokenizer.apply_chat_template(messages, tokenize=False))

<|user|>
The birch canoe slid on the smooth planks.<|end|>
<|assistant|>
On the smooth planks, the birch canoe slid. Yes, hrrrm.<|end|>
<|endoftext|>


# Fine-Tuning with SFTTrainer

## SFTConfig
* Memory Usage
* Dataset related
* Training parameters
* Environment and logging

In [14]:
from trl import SFTConfig

sft_config = SFTConfig(
    # Memory Group
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    gradient_accumulation_steps=1,
    per_device_train_batch_size=16,
    auto_find_batch_size=True,

    # Dataset related
    max_length=64,
    packing=True,
    packing_strategy="wrapped",

    # Training parameters
    num_train_epochs=10,
    learning_rate=3e-4,
    optim="paged_adamw_8bit",

    # Logging parameters
    logging_steps=10,
    logging_dir="./logs",
    output_dir="./phi3-mini-yoda-adapter",
    report_to="none",
    bf16=torch.cuda.is_bf16_supported(including_emulation=False)
)

## SFTTrainer

In [15]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    args=sft_config,
    train_dataset=dataset,
)

Tokenizing train dataset:   0%|          | 0/720 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/720 [00:00<?, ? examples/s]

In [16]:
dl = trainer.get_train_dataloader()
batch = next(iter(dl))

In [17]:
batch["input_ids"][0], batch["labels"][0]

(tensor([ 3974, 29892,  4337,   278,   325,   271, 29892,   366,  1818, 29889,
         32007, 32000, 32010,   450,   289,   935,   310,   278,   282,   457,
          5447,   471,   528,  4901,   322,  6501, 29889, 32007, 32001, 26399,
          1758,  4317, 29889,  1383,  4901,   322,  6501, 29892,   278,   289,
           935,   310,   278,   282,   457,  5447,   471, 29889, 32007, 32000,
         32010,   951,  5989,  2507, 17354,   322, 13328,   297,   278,  6416,
         29889, 32007, 32001,   512], device='cuda:0'),
 tensor([ 3974, 29892,  4337,   278,   325,   271, 29892,   366,  1818, 29889,
         32007, 32000, 32010,   450,   289,   935,   310,   278,   282,   457,
          5447,   471,   528,  4901,   322,  6501, 29889, 32007, 32001, 26399,
          1758,  4317, 29889,  1383,  4901,   322,  6501, 29892,   278,   289,
           935,   310,   278,   282,   457,  5447,   471, 29889, 32007, 32000,
         32010,   951,  5989,  2507, 17354,   322, 13328,   297,   278,  64

In [18]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 0}.


Step,Training Loss
10,2.855600
20,1.832200
30,1.606400
40,1.522500
50,1.400000
60,1.296000
70,1.194400
80,0.999300
90,0.901600
100,0.632300


TrainOutput(global_step=220, training_loss=0.8359539758075367, metrics={'train_runtime': 1613.0697, 'train_samples_per_second': 2.114, 'train_steps_per_second': 0.136, 'total_flos': 4890970340720640.0, 'train_loss': 0.8359539758075367, 'epoch': 10.0})

# Querying the Model

In [20]:
def gen_prompt(tokenizer, sentence):
  converted_sample = [{"role": "user", "content": sentence}]
  return tokenizer.apply_chat_template(converted_sample, tokenize=False, add_generation_prompt=True)

sentence = "The force is strong in you!"
prompt = gen_prompt(tokenizer, sentence)
print(prompt)

<|user|>
The force is strong in you!<|end|>
<|assistant|>



In [27]:
from contextlib import nullcontext
import torch

def generate(model, tokenizer, prompt, max_new_tokens=64, skip_special_tokens=False):
  tokenized_input = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
  tokenized_input = tokenized_input.to(model.device)
  model.eval()

  ctx = torch.autocast(device_type=model.device.type, dtype=model.dtype) if model.dtype in [torch.float16, torch.bfloat16] else nullcontext()

  with ctx:
    gen_output = model.generate(**tokenized_input, max_new_tokens=max_new_tokens, eos_token_id=tokenizer.eos_token_id)

  output = tokenizer.batch_decode(gen_output, skip_special_tokens=skip_special_tokens)
  return output[0]

In [28]:
print(generate(model, tokenizer, prompt))

<|user|> The force is strong in you!<|end|><|assistant|> Strong in you, the force is.<|end|><|endoftext|>


In [29]:
trainer.save_model("local-phi3-mini-yoda-adapter")

In [30]:
from huggingface_hub import login
login()

In [31]:
trainer.push_to_hub()

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...a-adapter/tokenizer.model: 100%|##########|  500kB /  500kB            

  ...adapter_model.safetensors:   0%|          | 27.8kB / 25.2MB            

  ...adapter/training_args.bin:   2%|2         |   153B / 6.22kB            

CommitInfo(commit_url='https://huggingface.co/Pigwen/phi3-mini-yoda-adapter/commit/2f04902b958525af22b9f766d55e110a77d304b2', commit_message='End of training', commit_description='', oid='2f04902b958525af22b9f766d55e110a77d304b2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Pigwen/phi3-mini-yoda-adapter', endpoint='https://huggingface.co', repo_type='model', repo_id='Pigwen/phi3-mini-yoda-adapter'), pr_revision=None, pr_num=None)